In [1]:
# Import dependencies
import pandas as pd
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read data
plays = pd.read_csv('../Resources/plays.csv')

week1 = pd.read_csv('../Resources/week1.csv')
week2 = pd.read_csv('../Resources/week2.csv')
week3 = pd.read_csv('../Resources/week3.csv')
week4 = pd.read_csv('../Resources/week4.csv')
week5 = pd.read_csv('../Resources/week5.csv')
week6 = pd.read_csv('../Resources/week6.csv')
week7 = pd.read_csv('../Resources/week7.csv')
week8 = pd.read_csv('../Resources/week8.csv')

tracking = week1.append([
    week2,
    week3,
    week4,
    week5,
    week6,
    week7,
    week8,
])

In [3]:
# Find tracking possession
player_tracking = tracking[tracking['team'] != 'football']
player_tracking = player_tracking.merge(plays[['gameId', 'playId', 'possessionTeam']])
player_tracking['possession'] = player_tracking['team'] == player_tracking['possessionTeam']

In [ ]:
# Get nearest player
opp_df = pd.DataFrame(columns=['gameId', 'playId', 'frameId', 'nflId', 'oppId'])
for i, r in player_tracking.iterrows():
    # Get opposition data
    if r['possession']:
        players = player_tracking.loc[
            (player_tracking['gameId'] == r['gameId']) & \
            (player_tracking['playId'] == r['playId']) & \
            (player_tracking['frameId'] == r['frameId'])
        ]
    else:
        players = player_tracking.loc[
            (player_tracking['gameId'] == r['gameId']) & \
            (player_tracking['playId'] == r['playId']) & \
            (player_tracking['frameId'] == r['frameId'])
        ]
    
    # Find distance to each opposing player
    players['x_diff'] = r['x'] - players['x']
    players['x_diff'] = players['x_diff']**2

    players['y_diff'] = r['y'] - players['y']
    players['y_diff'] = players['y_diff']**2

    players['diff_sum'] = players['x_diff'] = players['y_diff']
    players['distance_opp'] = players['diff_sum'].apply(lambda x: math.sqrt(x))
    
    # Append nearest player
    players.sort_values(by=['distance_opp'])
    opp_df = opp_df.append(pd.DataFrame({
        'gameId': players['gameId'].iloc[0],
        'playId': players['playId'].iloc[0],
        'frameId': players['frameId'].iloc[0],
        'nflId': r['nflId'],
        'oppId': players['nflId'].iloc[0]
    }, index=[0]))

In [ ]:
# Merge with tracking data
tracking_with_opp = tracking.merge(opp_df[['gameId', 'playId', 'frameId', 'nflId', 'oppId']], on=['gameId', 'playId', 'frameId', 'nflId'])

In [ ]:
# Save to CSV
tracking_with_opp.to_csv('tracking_with_opp.csv')